**The same task as Lab 1 except using `Word Embeddings OR RNNs`**

**Watch out!<br>
Preproceesing here is a a bit different<br>
Just take the cleaned data from the previous lab and cont...**


**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 60 Mins**

### Import Libraries

In [71]:
import spacy
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

import re

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import spacy
import unidecode
import contractions
from word2number import w2n

nlp = spacy.load('en_core_web_md')

from tqdm.auto import tqdm

import tensorflow as tf
import spacy

### Load Dataset

In [72]:
tweets = pd.read_csv('dataset.csv')
tweets['tweet']

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31957    ate @user isz that youuu?ðððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960    @user #sikh #temple vandalised in in #calgary,...
31961                     thank you @user for you follow  
Name: tweet, Length: 31962, dtype: object

### Preprocessing


In [73]:
def process_tweet(tweet):
    return " ".join(re.sub("([\@][A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())
 
def text_preprocessing(text):
    text = process_tweet(text)

    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
 
    text = text.strip()
    text =  " ".join(text.split())
 
    text = unidecode.unidecode(text)
 
    text = contractions.fix(text)
 
    text = text.lower()
 
    doc = nlp(text) 
    clean_text = []

    for token in doc:
        flag = True
        edit = token.text

        if token.is_stop and token.pos_ != 'NUM': 
            flag = False
#         if token.pos_ == 'PUNCT' and flag == True: 
#             flag = False
        if token.pos_ == 'SYM' and flag == True: 
            flag = False
        if (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        if token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        elif token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [74]:
tweets['tweet'] = tweets['tweet'].apply(lambda x : " ".join(text_preprocessing(x)))

In [75]:
# tweets['tweet'] = tweets['tweet'].apply(lambda x : " ".join(process_tweet(x)))
tweets['tweet']

0        father dysfunctional selfish drag kid dysfunct...
1        thank lyft credit t use don t offer wheelchair...
2                                           bihday majesty
3                                          model love time
4                            factsguide society motivation
                               ...                        
31957                                        eat isz youuu
31958    nina turner airwave try wrap mantle genuine he...
31959          listen sad song monday morning otw work sad
31960        sikh temple vandalise calgary wso condemn act
31961                                         thank follow
Name: tweet, Length: 31962, dtype: object

### Modelling
**Use any Advanced technique such as: word2vec, glove, RNNs ... etc**

* split the data

In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(tweets['tweet'],tweets['label'],
                                                   test_size=0.2, random_state=42,
                                                   stratify=tweets['label'])
print(len(X_train))
print(len(X_test))
type(X_train)

25569
6393


pandas.core.series.Series

In [77]:
max_sequence = 0

for sent in X_train:
    max_sequence = max(len(sent),max_sequence)

print(max_sequence)

129


* Vectorize the words

In [78]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=100 ,oov_token='UNK')
tok.fit_on_texts(X_train)

In [79]:
print(tok.word_index['UNK'])
print(len(tok.word_index))
length_of_word_index = len(tok.word_index)

1
29623


In [80]:
x_train_v = np.zeros((len(X_train), 300))
x_test_v = np.zeros((len(X_test), 300))

for i, doc in tqdm(enumerate(nlp.pipe(X_train)), total=len(X_train)):
    x_train_v[i, :] = doc.vector

for i, doc in tqdm(enumerate(nlp.pipe(X_test)), total=len(X_test)):
    x_test_v[i, :] = doc.vector

  0%|          | 0/25569 [00:00<?, ?it/s]

  0%|          | 0/6393 [00:00<?, ?it/s]

* padding

In [81]:
max_sequence_len = 100

x_train_padded = np.zeros((x_train_v.shape[0], max_sequence_len))

for i, sent in enumerate(x_train_v):
    x_train_padded[i, :len(sent)] = sent[:max_sequence_len]
x_train_padded.shape

(25569, 100)

In [82]:
x_test_padded = np.zeros((x_test_v.shape[0], max_sequence_len))

for i, sent in enumerate(x_test_v):
    x_test_padded[i, :len(sent)] = sent[:max_sequence_len]
x_test_padded.shape

(6393, 100)

* Build the model

In [83]:
VOCAB_SIZE = len(tweets)
VOCAB_SIZE

31962

In [84]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [85]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          2045568   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,058,049
Trainable params: 2,058,049
Non-trainable params: 0
_________________________________________________________________


In [86]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [87]:
x_train_padded.shape

(25569, 100)

In [1]:
x_train_padded_ = x_train_padded[0:100]
y_train_ = y_train[0:100]

NameError: name 'x_train_padded' is not defined

In [ ]:
history = model.fit(x_train_padded_, y_train_)

In [ ]:
kljewlkcewvwe

In [ ]:
test_loss, test_acc = model.evaluate(x_test_padded, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

#### Evaluation

#### Done!